## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-08-10-58-49 +0000,nypost,Mamdani failed to recruit enough emergency sno...,https://nypost.com/2026/02/08/us-news/mamdani-...
1,2026-02-08-10-40-53 +0000,startribune,Live: Lindsey Vonn set to race her last Olympi...,https://www.startribune.com/2026-winter-olympi...
2,2026-02-08-10-39-04 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/08/world/...
3,2026-02-08-10-30-00 +0000,wsj,The pace of hiring in the U.S. has dropped off...,https://www.wsj.com/economy/jobs/this-is-why-i...
4,2026-02-08-10-21-59 +0000,bbc,Polls close in Thailand election that pitted r...,https://www.bbc.com/news/articles/cx2jn4z4eq0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2414/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
65,trump,16
51,mayor,8
84,epstein,8
252,ice,8
127,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...,48
48,2026-02-08-03-03-15 +0000,nyt,"Savannah Guthrie, in New Video, Promises to Pa...",https://www.nytimes.com/2026/02/07/us/savannah...,44
136,2026-02-07-16-52-10 +0000,nypost,Tehran’s foreign minister warns they’re ready ...,https://nypost.com/2026/02/07/world-news/tehra...,43
113,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...,38
58,2026-02-08-01-58-07 +0000,bbc,US news anchor Savannah Guthrie says 'we will ...,https://www.bbc.com/news/articles/ce9y99eg0nno...,36


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,48,2026-02-08-00-12-00 +0000,wsj,Whistleblower complaint against Tulsi Gabbard ...,https://www.wsj.com/politics/national-security...
48,44,2026-02-08-03-03-15 +0000,nyt,"Savannah Guthrie, in New Video, Promises to Pa...",https://www.nytimes.com/2026/02/07/us/savannah...
62,36,2026-02-08-01-47-26 +0000,nypost,Radical LA councilmember Nithya Raman’s shocki...,https://nypost.com/2026/02/07/us-news/la-counc...
113,31,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...
114,29,2026-02-07-20-55-27 +0000,eltecolote,‘We’re not criminals’: Hot dog vendors say S.F...,https://eltecolote.org/content/en/sf-food-vend...
34,29,2026-02-08-05-28-53 +0000,nypost,Anti-ICE protesters attack their own supporter...,https://nypost.com/2026/02/08/us-news/anti-ice...
153,24,2026-02-07-13-41-15 +0000,cbc,"Lindsey Vonn completes another training run, r...",https://www.cbc.ca/sports/olympics/winter/alpi...
163,21,2026-02-07-12-00-00 +0000,latimes,San Bernardino County reports first measles ca...,https://www.latimes.com/california/story/2026-...
122,21,2026-02-07-19-42-07 +0000,nypost,Rick Caruso blasts Karen Bass — and drops trut...,https://nypost.com/2026/02/07/us-news/rick-car...
21,20,2026-02-08-09-55-25 +0000,nypost,Suspect who allegedly shot top Russian general...,https://nypost.com/2026/02/08/world-news/suspe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
